# More transformations in Pandas

In this lecture we will learn about a few critical transformations that are frequently used. Those are:

- Joins
- Pivot Table
- Group by

Part of the magic of using pandas for doing transformations is that aggregation functions for **Group By** can be custom. We will solve a problem which requires us to do that.

In addition to that, we will learn a bit about MultiIndex - which is something we have to know as it's automatically created when we do a **Pivot table** or a **Group By** transformation.

## Data

Our data today are stats about the air traffic in the US. We have two files:
- `airport-codes.csv` - this file contains metadata for all aiports in the world.
- `us_air_transport_stats.csv` - this file contains the stats like number of passengers, amount of freight etc. for each airport origin and destination and each airline carrier.

In [1]:
import pandas as pd

In [2]:
codes_df = pd.read_csv('airport-codes.csv')
del codes_df['continent']
del codes_df['iata_code']
codes_df = codes_df[codes_df['type'] != 'closed']
codes_df.head(10)

,ident,type,name,elevation_ft,iso_country,iso_region,municipality,gps_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,US,US-PA,Bensalem,00A,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,US,US-KS,Leoti,00AA,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,US,US-AK,Anchor Point,00AK,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,US,US-AL,Harvest,00AL,00AL,"-86.77030181884766, 34.86479949951172"
5,00AS,small_airport,Fulton Airport,1100.0,US,US-OK,Alex,00AS,00AS,"-97.8180194, 34.9428028"
6,00AZ,small_airport,Cordes Airport,3810.0,US,US-AZ,Cordes,00AZ,00AZ,"-112.16500091552734, 34.305599212646484"
7,00CA,small_airport,Goldstone /Gts/ Airport,3038.0,US,US-CA,Barstow,00CA,00CA,"-116.888000488, 35.350498199499995"
8,00CL,small_airport,Williams Ag Airport,87.0,US,US-CA,Biggs,00CL,00CL,"-121.763427, 39.427188"
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350.0,US,US-CA,Pine Valley,00CN,00CN,"-116.4597417, 32.7273736"
11,00FA,small_airport,Grass Patch Airport,53.0,US,US-FL,Bushnell,00FA,00FA,"-82.21900177001953, 28.64550018310547"


In [3]:
stats_df = pd.read_csv('us_air_transport_stats.csv')
stats_df.head(10)

,PASSENGERS,FREIGHT,MAIL,DISTANCE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,YEAR,MONTH
0,0,0,0,156,0MQ,21253.0,0MQ,"Multi-Aero, Inc. d/b/a Air Choice One",13930.0,ORD,11288.0,DEC,2018,9
1,0,0,0,110,0MQ,21253.0,0MQ,"Multi-Aero, Inc. d/b/a Air Choice One",15016.0,STL,11288.0,DEC,2018,9
2,0,0,0,949,0WQ,21352.0,0WQ,Avjet Corporation,10279.0,AMA,10800.0,BUR,2018,9
3,0,0,0,725,0WQ,21352.0,0WQ,Avjet Corporation,10372.0,ASE,10800.0,BUR,2018,9
4,0,0,0,18,0WQ,21352.0,0WQ,Avjet Corporation,10800.0,BUR,12892.0,LAX,2018,9
5,0,0,0,303,0WQ,21352.0,0WQ,Avjet Corporation,10800.0,BUR,13771.0,NUQ,2018,9
6,0,0,0,86,0WQ,21352.0,0WQ,Avjet Corporation,10800.0,BUR,14689.0,SBA,2018,9
7,0,0,0,326,0WQ,21352.0,0WQ,Avjet Corporation,10800.0,BUR,14771.0,SFO,2018,9
8,0,0,0,46,0WQ,21352.0,0WQ,Avjet Corporation,10800.0,BUR,14908.0,SNA,2018,9
9,0,0,0,678,0WQ,21352.0,0WQ,Avjet Corporation,10800.0,BUR,15041.0,SUN,2018,9


In [4]:
stats_df['ORIGIN'].nunique()

1249

The US air traffic stats table contains data for 1249 airports.

## Join and Pivot

We are going to look at the Join and Pivot operations first. We are going to work through a problem statement which will require us to perform these operations.

### Problem statement

We want to create a table that compares the number of passengers flown by each major airline out of each large airports in the US. Our final table should have one row for each airport, and one column for each carrier. We will eventually limit the number of columns by selecting only the large carriers.

Here are the steps we need to perform to be able to do that:

1. Identify all large airports in the US from airport codes dataframe.
2. Inner join the stats df with the dataframe containing only the large airports.
3. Pivot on the origin airport code to get the stats for each airline in different columns.
4. Filter only large airlines.

#### Step 1

In [5]:
codes_df['type'].unique()

array(['heliport', 'small_airport', 'seaplane_base', 'balloonport',
       'medium_airport', 'large_airport'], dtype=object)

In [6]:
large_airports_df = codes_df[(codes_df['type'] == 'large_airport') & (codes_df['iso_country'] == 'US')][['iso_region', 'name', 'municipality', 'local_code']]
large_airports_df.head(10)

,iso_region,name,municipality,local_code
25914,US-NM,Albuquerque International Sunport,Albuquerque,ABQ
25933,US-MD,Andrews Air Force Base,Camp Springs,ADW
25946,US-TX,Fort Worth Alliance Airport,Fort Worth,AFW
25950,US-GA,Augusta Regional At Bush Field,Augusta,AGS
25983,US-TX,Rick Husband Amarillo International Airport,Amarillo,AMA
26035,US-GA,Hartsfield Jackson Atlanta International Airport,Atlanta,ATL
26044,US-TX,Austin Bergstrom International Airport,Austin,AUS
26048,US-NC,Asheville Regional Airport,Asheville,AVL
26072,US-CA,Beale Air Force Base,Marysville,BAB
26073,US-LA,Barksdale Air Force Base,Bossier City,BAD


The table has the column `type` which tells us if an airport is a `large_airport` or not. We filter on that (and include only US airports, as our air traffic stats are only for US), and then select only relevant columns.

#### Step 2

Now we need to join the dataframe we just created with the air traffic stats dataframe. In `pandas`, the function `merge()` is used to join two dataframes. It has the following arguments:
- `left` and `right` - these are the two dataframes that you want to join
- `left_on` - this is the column in the left table that has a foreign key into the right dataframe. This can be a list of columns.
- `right_on` - the symmetrical counterpart to `left_on`. 
- `how` - the kind of join you want.

The output will contain every column in the left table, as well as every column in the right table.

The [API reference page for `pandas.merge()` here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html) has more details.

In the next cell, we use the `merge()` function to do the join we outlined in step 2.

In [7]:
large_airport_stats_df = pd.merge(stats_df, large_airports_df, left_on='ORIGIN', right_on='local_code', how='inner')
large_airport_stats_df.head(10)

,PASSENGERS,FREIGHT,MAIL,DISTANCE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,YEAR,MONTH,iso_region,name,municipality,local_code
0,0,0,0,156,0MQ,21253.0,0MQ,"Multi-Aero, Inc. d/b/a Air Choice One",13930.0,ORD,11288.0,DEC,2018,9,US-IL,Chicago O'Hare International Airport,Chicago,ORD
1,0,1309768,0,2846,KAQ,20370.0,KAQ,Kalitta Air LLC,13930.0,ORD,10299.0,ANC,2018,9,US-IL,Chicago O'Hare International Airport,Chicago,ORD
2,0,748398,0,606,KAQ,20370.0,KAQ,Kalitta Air LLC,13930.0,ORD,10397.0,ATL,2018,9,US-IL,Chicago O'Hare International Airport,Chicago,ORD
3,0,1476087,0,264,KAQ,20370.0,KAQ,Kalitta Air LLC,13930.0,ORD,11193.0,CVG,2018,9,US-IL,Chicago O'Hare International Airport,Chicago,ORD
4,0,397114,0,740,KAQ,20370.0,KAQ,Kalitta Air LLC,13930.0,ORD,12478.0,JFK,2018,9,US-IL,Chicago O'Hare International Airport,Chicago,ORD
5,0,196912,0,1197,KAQ,20370.0,KAQ,Kalitta Air LLC,13930.0,ORD,13303.0,MIA,2018,9,US-IL,Chicago O'Hare International Airport,Chicago,ORD
6,0,0,0,1846,KAQ,20370.0,KAQ,Kalitta Air LLC,13930.0,ORD,14771.0,SFO,2018,9,US-IL,Chicago O'Hare International Airport,Chicago,ORD
7,0,1532654,0,2846,KD,21629.0,KD,Western Global,13930.0,ORD,10299.0,ANC,2018,9,US-IL,Chicago O'Hare International Airport,Chicago,ORD
8,0,0,0,0,KD,21629.0,KD,Western Global,13930.0,ORD,13930.0,ORD,2018,9,US-IL,Chicago O'Hare International Airport,Chicago,ORD
9,0,1594787,0,286,KD,21629.0,KD,Western Global,13930.0,ORD,14730.0,SDF,2018,9,US-IL,Chicago O'Hare International Airport,Chicago,ORD


In [8]:
large_airport_stats_df['ORIGIN'].nunique()

157

The join gives us a table where the `ORIGIN` column now only contains the 157 large airports. Note that the `DEST` column still contains all the airports, not just the large ones.

#### Step 3

The pivot table is the other new transformation required to solve our problem. 

- Pivoting a table is a transformation in which you convert one column in the input table to many different columns in the output table, one for each value in that column in te input table. 
- For example, our input table (`large_airport_stats_df`) has a column for the carrier. In our output, we want a column for each of the carriers. Thus, if we pivot the table on that column, we will get our output. 
- There are two other types of columns in the pivot table transformation:p
  - **Row columns** - these are the columns that continue to have all their values present in one column. In our problem requirement, the Origin airport is an example of this type of column.
  - **Value columns** - these are the measures that will get aggregated as you do the transformation. For example, the number of passengers

In our earlier SQL lectures, we saw how the pivot table transformation can be implemented using the `GROUP BY` clause in SQL. Specifically, we group by the **row Columns**, and then we can create a row which contains a column for each value in the pivot column. Thus, the **pivot table transformation is inherently a group by transformation**.

In `pandas`, the function `pivot_table()` implements the pivot table transformation. It's arguments are:
- `index` - you provide the row columns here. In the output, these columns become part of the dataframe index (more on this later).
- `columns` - this is the list of pivot columns. If you have more than one column here, then the output will have a column for _every combination_ of values in the pivot columns.
- `values` - these are the value columns.
- `aggfunc` - The aggregate function to apply in the pivot table transformation. This can be any python function which can take a Series as an argument.

The [API reference page for `pandas.pivot_table()` here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html) has more details.

We use this function below. The `fill_value` function replaces any `NaN` value with a 0. Why might you get a `NaN`?

In [9]:
pivot_df = pd.pivot_table(large_airport_stats_df, 
                          index=['ORIGIN', 'iso_region', 'name', 'municipality'], 
                          columns=['CARRIER_NAME'], 
                          values='PASSENGERS', 
                          aggfunc=sum,
                          fill_value=0)
pivot_df.head(10)

,,,CARRIER_NAME,40-Mile Air,ABX Air Inc,"ADVANCED AIR, LLC",Aerodynamics Inc. d/b/a SkyValue d/b/a SkyValue Airways,Air Transport International,Air Wisconsin Airlines Corp,Alaska Airlines Inc.,Alaska Central Express,Allegiant Air,Aloha Air Cargo,...,USA Jet Airlines Inc.,Ultimate JetCharters LLC dba Ultimate Air Shuttle,United Air Lines Inc.,United Parcel Service,Via Airlines d/b/a Charter Air Transport,Virgin America,Warbelow,Western Global,Wright Air Service,XTRA Airways
ORIGIN,iso_region,name,municipality,,,,,,,,,,,,,,,,,,,,,
ABQ,US-NM,Albuquerque International Sunport,Albuquerque,0,0,0,0,0,0,33682,0,28698,0,...,0,0,108961,0,0,0,0,0,0,15
ADW,US-MD,Andrews Air Force Base,Camp Springs,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AFW,US-TX,Fort Worth Alliance Airport,Fort Worth,0,0,0,29,0,0,0,0,804,0,...,0,0,108,0,0,0,0,0,0,0
AGS,US-GA,Augusta Regional At Bush Field,Augusta,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,389
AMA,US-TX,Rick Husband Amarillo International Airport,Amarillo,0,0,13,1,0,0,0,0,0,0,...,0,0,0,0,910,0,0,0,0,0
ANC,US-AK,Ted Stevens Anchorage International Airport,Anchorage,4,0,0,0,0,0,1707632,2569,0,0,...,0,0,135994,0,0,0,0,0,0,0
ATL,US-GA,Hartsfield Jackson Atlanta International Airport,Atlanta,0,0,6,70,0,0,101508,0,528,0,...,0,0,546920,0,0,0,0,0,0,391
AUS,US-TX,Austin Bergstrom International Airport,Austin,0,0,0,0,24,0,207375,0,139113,0,...,0,0,919824,0,11487,17456,0,0,0,0
AVL,US-NC,Asheville Regional Airport,Asheville,0,0,0,0,0,0,0,0,212253,0,...,0,0,23273,0,0,0,0,0,0,0


In [10]:
len(pivot_df.columns)

99

A couple of things to note in the output:
- The index (columns whose values are in bold) contains many columns now - exactly the ones we specified in the `index` argument in the pivot_table() function.
- There are 99 columns (and these don't include the ones in the index) in the output dataframe, one for each carrier.

#### Step 4

- Our last step is prune the number of columns to include only the large carriers in the columns. 
- A large carrier can be identified by the total number of passengers it ferries, from all origin airports. We can say that an airline transporting more than 10 Million passengers in an year is large.
- This problem is similar to a problem of filtering rows, but it's transposed - in this case we have to filter columns. 
- In the solution, we don't use any new function to achieve this. Can you think on your own before seeing the solution?

In [11]:
imp_airlines = pivot_df.columns[pivot_df.apply(lambda x: sum(x) > 10 * 1000000, axis=0)]
imp_airlines

Index(['Alaska Airlines Inc.', 'American Airlines Inc.',
       'Delta Air Lines Inc.', 'Endeavor Air Inc.', 'Envoy Air',
       'Frontier Airlines Inc.', 'JetBlue Airways', 'Mesa Airlines Inc.',
       'PSA Airlines Inc.', 'Republic Airline', 'SkyWest Airlines Inc.',
       'Southwest Airlines Co.', 'Spirit Air Lines', 'United Air Lines Inc.'],
      dtype='object', name='CARRIER_NAME')

In [12]:
airlines_compare_df = pivot_df[imp_airlines]
airlines_compare_df.head(10)

,,,CARRIER_NAME,Alaska Airlines Inc.,American Airlines Inc.,Delta Air Lines Inc.,Endeavor Air Inc.,Envoy Air,Frontier Airlines Inc.,JetBlue Airways,Mesa Airlines Inc.,PSA Airlines Inc.,Republic Airline,SkyWest Airlines Inc.,Southwest Airlines Co.,Spirit Air Lines,United Air Lines Inc.
ORIGIN,iso_region,name,municipality,,,,,,,,,,,,,,
ABQ,US-NM,Albuquerque International Sunport,Albuquerque,33682,307425,191373,0,7301,54361,42836,178666,0,25454,146675,1282912,0,108961
ADW,US-MD,Andrews Air Force Base,Camp Springs,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AFW,US-TX,Fort Worth Alliance Airport,Fort Worth,0,0,0,0,0,0,0,0,0,0,0,0,0,108
AGS,US-GA,Augusta Regional At Bush Field,Augusta,0,0,97318,54472,970,0,0,5,86523,1032,32411,0,0,0
AMA,US-TX,Rick Husband Amarillo International Airport,Amarillo,0,291,0,0,55964,0,0,77298,0,0,541,164082,0,0
ANC,US-AK,Ted Stevens Anchorage International Airport,Anchorage,1707632,37168,317423,0,0,0,24814,0,0,0,0,0,0,135994
ATL,US-GA,Hartsfield Jackson Atlanta International Airport,Atlanta,101508,1270635,33239588,1472242,40113,530481,367303,154716,59166,293761,1058641,4890261,1119869,546920
AUS,US-TX,Austin Bergstrom International Airport,Austin,207375,1321041,974791,21174,4055,524069,282721,87094,2314,65473,174239,2729373,0,919824
AVL,US-NC,Asheville Regional Airport,Asheville,0,180,57854,38273,0,0,0,0,117489,0,87677,0,12316,23273


The dataframe shown above satisfies the problem statement.

## Multi Index

As noted before, our index in the final result of previous section has four different columns in the index. This type of index is called a `MultiIndex`, which is actually a class in `pandas`.

In this section, we will learn how to work with a `MultiIndex`. It's important to know about this because a `MultiIndex` is automatically created when you have multiple columns in the `index` argument of the `pivot_table()` call, or in `groupby()` call, as will see in next section.

We can see below the type of the index for the `airlines_compare_df` that we just created.

In [55]:
type(airlines_compare_df.index)

pandas.core.indexes.multi.MultiIndex

The "columns" of a `MultiIndex` are called _levels_. 

- The levels of a `MultiIndex` are ordered, and the first one (`ORIGIN` in this case) is called level 0, the second one (`iso_region`) is called level 1, and so on.
- The `names` attribute index shows us the name of all the levels.

In [60]:
airlines_compare_df.index.names

FrozenList(['ORIGIN', 'iso_region', 'name', 'municipality'])

The first thing we want to know is how do we select a part of the dataframe using the values of the index. We have used the `loc()` method before to do that, and we can do the same here, as shown below:

In [59]:
airlines_compare_df.loc[('ABQ', 'US-NM')]

,CARRIER_NAME,Alaska Airlines Inc.,American Airlines Inc.,Delta Air Lines Inc.,Endeavor Air Inc.,Envoy Air,Frontier Airlines Inc.,JetBlue Airways,Mesa Airlines Inc.,PSA Airlines Inc.,Republic Airline,SkyWest Airlines Inc.,Southwest Airlines Co.,Spirit Air Lines,United Air Lines Inc.
name,municipality,,,,,,,,,,,,,,
Albuquerque International Sunport,Albuquerque,33682,307425,191373,0,7301,54361,42836,178666,0,25454,146675,1282912,0,108961


A few things to note here:
- We have provided values for the first two levels of the index, as a tuple to the `loc()` method. That selects the dataframe for us where the values for those two levels matches the ones provided.
- You can provide values for any number of levels in the `MultiIndex`, but it must go from first level to higher levels as the order of values in the tuple.

Now let's look at other things we can do with a `MultiIndex`.

- If you look at the dataframe index, you will notice that the order of levels is not quite right.
- A proper index should have the largest granularity level as the first level, and so on.
- In our case, we see that `iso_region` has the largest granularity, as a state can have many airports.
- The `municipality` should be the next level as one city can have multiple airports servicing it (like JFK and La Guardia for New York City).

There is a `DataFrame` method called `reorder_levels()` which can reorder the levels of the `MultiIndex` for us. Note that in the call shown below, we use `axis=0` as one of the arguments - this is to specify that we are referring to the "row index", and not the columns.

Note that the current, incorrect, order was defined by the order in which we listed the column names to the `index` argument to the `pivot_table()` function. 

In [64]:
final_airline_compare = airlines_compare_df.reorder_levels([1,3,0,2], axis=0)
final_airline_compare.head(5)

,,,CARRIER_NAME,Alaska Airlines Inc.,American Airlines Inc.,Delta Air Lines Inc.,Endeavor Air Inc.,Envoy Air,Frontier Airlines Inc.,JetBlue Airways,Mesa Airlines Inc.,PSA Airlines Inc.,Republic Airline,SkyWest Airlines Inc.,Southwest Airlines Co.,Spirit Air Lines,United Air Lines Inc.
iso_region,municipality,ORIGIN,name,,,,,,,,,,,,,,
US-NM,Albuquerque,ABQ,Albuquerque International Sunport,33682,307425,191373,0,7301,54361,42836,178666,0,25454,146675,1282912,0,108961
US-MD,Camp Springs,ADW,Andrews Air Force Base,0,0,0,0,0,0,0,0,0,0,0,0,0,0
US-TX,Fort Worth,AFW,Fort Worth Alliance Airport,0,0,0,0,0,0,0,0,0,0,0,0,0,108
US-GA,Augusta,AGS,Augusta Regional At Bush Field,0,0,97318,54472,970,0,0,5,86523,1032,32411,0,0,0
US-TX,Amarillo,AMA,Rick Husband Amarillo International Airport,0,291,0,0,55964,0,0,77298,0,0,541,164082,0,0


Let's see how we see the data for all airports in the New York city.

In [66]:
final_airline_compare.loc[('US-NY', 'New York')]

/home/aman/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,CARRIER_NAME,Alaska Airlines Inc.,American Airlines Inc.,Delta Air Lines Inc.,Endeavor Air Inc.,Envoy Air,Frontier Airlines Inc.,JetBlue Airways,Mesa Airlines Inc.,PSA Airlines Inc.,Republic Airline,SkyWest Airlines Inc.,Southwest Airlines Co.,Spirit Air Lines,United Air Lines Inc.
ORIGIN,name,,,,,,,,,,,,,,
JFK,John F Kennedy International Airport,555773,2017625,4275329,1263384,162502,0,5174316,0,15518,145377,86827,0,0,108
LGA,La Guardia Airport,0,2597812,3004050,1427111,452801,167096,623309,88859,63438,1678189,437635,1372406,654418,1115610


Even though the data now has index levels in the right order, their values are not sorted. The first row is for the state of New Mexico, but the next row goes to the state of Maryland, instead of showing the other airports in New Mexico.

To sort a `MultiIndex`, we can use the `level` argument to the `sort_index()` method of the `DataFrame`.

In [68]:
final_airline_compare.sort_index(level=[0, 1, 2], inplace=True)
final_airline_compare.head(10)

CARRIER_NAME                                                                       Alaska Airlines Inc.  \
iso_region municipality ORIGIN name                                                                       
US-AK      Anchorage    ANC    Ted Stevens Anchorage International Airport                      1707632   
           Fairbanks    FAI    Fairbanks International Airport                                   389930   
US-AL      Birmingham   BHM    Birmingham-Shuttlesworth International Airport                         0   
           Huntsville   HSV    Huntsville International Carl T Jones Field                            0   
           Mobile       MOB    Mobile Regional Airport                                                0   
           Montgomery   MGM    Montgomery Regional (Dannelly Field) Airport                           0   
US-AR      Fort Smith   FSM    Fort Smith Regional Airport                                            0   
           Little Rock  LIT    Bill & Hillary Clinton National Airport/Adams F...                     0   
US-AZ      Phoenix      PHX    Phoenix Sky Harbor International Airport                          417780   
           Tucson       TUS    Tucson International Airport                                       58151   

CARRIER_NAME                                                                       American Airlines Inc.  \
iso_region municipality ORIGIN name                                                                         
US-AK      Anchorage    ANC    Ted Stevens Anchorage International Airport                          37168   
           Fairbanks    FAI    Fairbanks International Airport                                          0   
US-AL      Birmingham   BHM    Birmingham-Shuttlesworth International Airport                         136   
           Huntsville   HSV    Huntsville International Carl T Jones Field                              0   
           Mobile       MOB    Mobile Regional Airport                                                  0   
           Montgomery   MGM    Montgomery Regional (Dannelly Field) Airport                             0   
US-AR      Fort Smith   FSM    Fort Smith Regional Airport                                              0   
           Little Rock  LIT    Bill & Hillary Clinton National Airport/Adams F...                    2609   
US-AZ      Phoenix      PHX    Phoenix Sky Harbor International Airport                           7626835   
           Tucson       TUS    Tucson International Airport                                        463881   

CARRIER_NAME                                                                       Delta Air Lines Inc.  \
iso_region municipality ORIGIN name                                                                       
US-AK      Anchorage    ANC    Ted Stevens Anchorage International Airport                       317423   
           Fairbanks    FAI    Fairbanks International Airport                                    66385   
US-AL      Birmingham   BHM    Birmingham-Shuttlesworth International Airport                    392466   
           Huntsville   HSV    Huntsville International Carl T Jones Field                       227371   
           Mobile       MOB    Mobile Regional Airport                                            38394   
           Montgomery   MGM    Montgomery Regional (Dannelly Field) Airport                         309   
US-AR      Fort Smith   FSM    Fort Smith Regional Airport                                            0   
           Little Rock  LIT    Bill & Hillary Clinton National Airport/Adams F...                255105   
US-AZ      Phoenix      PHX    Phoenix Sky Harbor International Airport                         1195311   
           Tucson       TUS    Tucson International Airport                                      138626   

CARRIER_NAME                                                                       Endeavor Air Inc.  \
iso_region munici

- The final transformation that we want to do to our index is the removal of one of the levels from the `MultiIndex` itself. 
- Once the airport code is specified (level = `ORIGIN`), the name of the airport is fixed. Thus, to have the airport name in the index is redundant.
- The `reset_index()` method in the `DataFrame` class takes an argument called `level`, which then removes that level from the `MultiIndex`.

In [69]:
final_airline_compare.reset_index(level=3, inplace=True)
final_airline_compare.head(10)

CARRIER_NAME                                                                 name  \
iso_region municipality ORIGIN                                                      
US-AK      Anchorage    ANC           Ted Stevens Anchorage International Airport   
           Fairbanks    FAI                       Fairbanks International Airport   
US-AL      Birmingham   BHM        Birmingham-Shuttlesworth International Airport   
           Huntsville   HSV           Huntsville International Carl T Jones Field   
           Mobile       MOB                               Mobile Regional Airport   
           Montgomery   MGM          Montgomery Regional (Dannelly Field) Airport   
US-AR      Fort Smith   FSM                           Fort Smith Regional Airport   
           Little Rock  LIT     Bill & Hillary Clinton National Airport/Adams ...   
US-AZ      Phoenix      PHX              Phoenix Sky Harbor International Airport   
           Tucson       TUS                          Tucson International Airport   

CARRIER_NAME                    Alaska Airlines Inc.  American Airlines Inc.  \
iso_region municipality ORIGIN                                                 
US-AK      Anchorage    ANC                  1707632                   37168   
           Fairbanks    FAI                   389930                       0   
US-AL      Birmingham   BHM                        0                     136   
           Huntsville   HSV                        0                       0   
           Mobile       MOB                        0                       0   
           Montgomery   MGM                        0                       0   
US-AR      Fort Smith   FSM                        0                       0   
           Little Rock  LIT                        0                    2609   
US-AZ      Phoenix      PHX                   417780                 7626835   
           Tucson       TUS                    58151                  463881   

CARRIER_NAME                    Delta Air Lines Inc.  Endeavor Air Inc.  \
iso_region municipality ORIGIN                                            
US-AK      Anchorage    ANC                   317423                  0   
           Fairbanks    FAI                    66385                  0   
US-AL      Birmingham   BHM                   392466              27377   
           Huntsville   HSV                   227371              11674   
           Mobile       MOB                    38394              17116   
           Montgomery   MGM                      309              35608   
US-AR      Fort Smith   FSM                        0              10524   
           Little Rock  LIT                   255105               1990   
US-AZ      Phoenix      PHX                  1195311                  0   
           Tucson       TUS                   138626                  0   

CARRIER_NAME                    Envoy Air  Frontier Airlines Inc.  \
iso_region municipality ORIGIN                                      
US-AK      Anchorage    ANC             0                       0   
           Fairbanks    FAI             0                       0   
US-AL      Birmingham   BHM         58441                   30553   
           Huntsville   HSV          8391                    8915   
           Mobile       MOB            56                       0   
           Montgomery   MGM         31549                       0   
US-AR      Fort Smith   FSM         20391                       0   
           Little Rock  LIT        214986                   22605   
US-AZ      Phoenix      PHX            50                  341282   
           Tucson       TUS             0                    4766   

CARRIER_NAME                    JetBlue Airways  Mesa Airlines Inc.  \
iso_region municipality ORIGIN                                        
US-AK      Anchorage    ANC               24814                   0   
           Fairbanks    FAI                   0                   0   
US-A

Since the columns of a dataframe are also defined by the `Index` type, it can also take the form of a `MultiIndex`.

Let's below how we might get a dataframe with a column multi-index.

In [70]:
pivot_df2 = pd.pivot_table(large_airport_stats_df, index=['ORIGIN', 'iso_region', 'name', 'municipality'], columns=['CARRIER_NAME'], values=['PASSENGERS', 'FREIGHT'], aggfunc=sum, fill_value=0)
pivot_df2.head(5)

FREIGHT  \
CARRIER_NAME                                                               40-Mile Air   
ORIGIN iso_region name                                        municipality               
ABQ    US-NM      Albuquerque International Sunport           Albuquerque            0   
ADW    US-MD      Andrews Air Force Base                      Camp Springs           0   
AFW    US-TX      Fort Worth Alliance Airport                 Fort Worth             0   
AGS    US-GA      Augusta Regional At Bush Field              Augusta                0   
AMA    US-TX      Rick Husband Amarillo International Airport Amarillo               0   

                                                                                        \
CARRIER_NAME                                                               ABX Air Inc   
ORIGIN iso_region name                                        municipality               
ABQ    US-NM      Albuquerque International Sunport           Albuquerque            0   
ADW    US-MD      Andrews Air Force Base                      Camp Springs           0   
AFW    US-TX      Fort Worth Alliance Airport                 Fort Worth             0   
AGS    US-GA      Augusta Regional At Bush Field              Augusta                0   
AMA    US-TX      Rick Husband Amarillo International Airport Amarillo               0   

                                                                                              \
CARRIER_NAME                                                               ADVANCED AIR, LLC   
ORIGIN iso_region name                                        municipality                     
ABQ    US-NM      Albuquerque International Sunport           Albuquerque                  0   
ADW    US-MD      Andrews Air Force Base                      Camp Springs                 0   
AFW    US-TX      Fort Worth Alliance Airport                 Fort Worth                   0   
AGS    US-GA      Augusta Regional At Bush Field              Augusta                      0   
AMA    US-TX      Rick Husband Amarillo International Airport Amarillo                     0   

                                                                                                                                    \
CARRIER_NAME                                                               Aerodynamics Inc. d/b/a SkyValue d/b/a SkyValue Airways   
ORIGIN iso_region name                                        municipality                                                           
ABQ    US-NM      Albuquerque International Sunport           Albuquerque                                                   0        
ADW    US-MD      Andrews Air Force Base                      Camp Springs                                                  0        
AFW    US-TX      Fort Worth Alliance Airport                 Fort Worth                                                    0        
AGS    US-GA      Augusta Regional At Bush Field              Augusta                                                       0        
AMA    US-TX      Rick Husband Amarillo International Airport Amarillo                                                      0        

                                                                                                        \
CARRIER_NAME                                                               Air Transport International   
ORIGIN iso_region name                                        municipality                               
ABQ    US-NM      Albuquerque International Sunport           Albuquerque                            0   
ADW    US-MD      Andrews Air Force Base                      Camp Springs                           0   
AFW    US-TX      Fort Worth Alliance Airport                 Fort Worth                             0   
AGS    US-GA      Augusta Regional At Bush Field              Augusta                                0   
AMA    US-TX      Rick Husband Amarillo International

In [71]:
type(pivot_df2.columns)

pandas.core.indexes.multi.MultiIndex

In [75]:
# The first level
pivot_df2.columns.levels[0]

Index(['FREIGHT', 'PASSENGERS'], dtype='object')

In [78]:
# The second level
pivot_df2.columns.levels[1][:10]  # Truncated to reduce clutter

Index(['40-Mile Air', 'ABX Air Inc', 'ADVANCED AIR, LLC',
       'Aerodynamics Inc. d/b/a SkyValue d/b/a SkyValue Airways',
       'Air Transport International', 'Air Wisconsin Airlines Corp',
       'Alaska Airlines Inc.', 'Alaska Central Express', 'Allegiant Air',
       'Aloha Air Cargo'],
      dtype='object', name='CARRIER_NAME')

- To access a specific column, we provide the tuple of index values in the same way as we did for the row index. 
- The only difference is that we don't need to use the `loc()` method to access a column.

In [80]:
pivot_df2[('FREIGHT', 'United Parcel Service')][:10]

ORIGIN  iso_region  name                                              municipality
ABQ     US-NM       Albuquerque International Sunport                 Albuquerque      39087941
ADW     US-MD       Andrews Air Force Base                            Camp Springs            0
AFW     US-TX       Fort Worth Alliance Airport                       Fort Worth              0
AGS     US-GA       Augusta Regional At Bush Field                    Augusta                 0
AMA     US-TX       Rick Husband Amarillo International Airport       Amarillo                0
ANC     US-AK       Ted Stevens Anchorage International Airport       Anchorage       523298938
ATL     US-GA       Hartsfield Jackson Atlanta International Airport  Atlanta          47303207
AUS     US-TX       Austin Bergstrom International Airport            Austin           18355109
AVL     US-NC       Asheville Regional Airport                        Asheville               0
BAB     US-CA       Beale Air Force Base             

- A second convenient way to achieve the same thing is to use the syntax `pivot_df2['FREIGHT']['United Parcel Service']`, which will work in a manner identical to what's shown above.

## Group By

Grouping the data into multiple groups, and applying aggregate functions to those groups is one of the most commonly used transformations in data analysis. We have looked at in detail in the SQL portion of our bootcamp - here we see how to do it in Python.

- For most real life scenarios, this transformation is quite easy to perform.
- But, I also want to cover a more advanced use case scenario - which is the case when we can use our own custom aggregate functions in group by. That is shown in Problem 2.


Let's solve a couple of problems to helps us understand the `DataFrame`'s `groupby()` method by better.

### Problem 1

- Find the passenger count and freight load for each route among the large airports in the US.
- Find the busiest routes for passengers and freights in the US.

What do we need to solve this problem?

1. We want to look at routes among the large airports of US. The dataframe that we used for our last problem, `large_airport_stats_df` has only the large airports in the `ORIGIN` column, but every single airport present in the `DEST` column. We need to do another join with the `large_airports_df` to truncate the number of destination airports.

2. We need to group by the origin and destination columns, so that we form groups for each route. Then we need to aggregate the `PASSENGER` and `FREIGHT` columns by summing them within each group.

3. To solve the second part of this problem, we need to sort the dataframe by passenger count or freight volume.

#### Step 1

In [82]:
large_route_stats = pd.merge(large_airport_stats_df, large_airports_df, left_on='DEST', right_on='local_code')
large_route_stats.head(5)

,PASSENGERS,FREIGHT,MAIL,DISTANCE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN,...,YEAR,MONTH,iso_region_x,name_x,municipality_x,local_code_x,iso_region_y,name_y,municipality_y,local_code_y
0,0,1309768,0,2846,KAQ,20370.0,KAQ,Kalitta Air LLC,13930.0,ORD,...,2018,9,US-IL,Chicago O'Hare International Airport,Chicago,ORD,US-AK,Ted Stevens Anchorage International Airport,Anchorage,ANC
1,0,1532654,0,2846,KD,21629.0,KD,Western Global,13930.0,ORD,...,2018,9,US-IL,Chicago O'Hare International Airport,Chicago,ORD,US-AK,Ted Stevens Anchorage International Airport,Anchorage,ANC
2,0,1011342,0,2846,PO,20100.0,PO,Polar Air Cargo Airways,13930.0,ORD,...,2018,9,US-IL,Chicago O'Hare International Airport,Chicago,ORD,US-AK,Ted Stevens Anchorage International Airport,Anchorage,ANC
3,0,2687090,0,2846,5Y,20007.0,5Y,Atlas Air Inc.,13930.0,ORD,...,2018,1,US-IL,Chicago O'Hare International Airport,Chicago,ORD,US-AK,Ted Stevens Anchorage International Airport,Anchorage,ANC
4,0,1494827,0,2846,5Y,20007.0,5Y,Atlas Air Inc.,13930.0,ORD,...,2018,2,US-IL,Chicago O'Hare International Airport,Chicago,ORD,US-AK,Ted Stevens Anchorage International Airport,Anchorage,ANC


- The join using the `merge()` function is pretty similar to before.
- If you look at the rightmost columns, you will see columns like `iso_region_x` and `municipality_y`. 
  - These are the columns which get added from `large_airports_df` when the join is done.
  - We have done two joins with the same table, thus the exact same columns get added to our table after the join.
  - To differentiate the names in this case, `pandas` automatically appends `_x` and `_y` to the names of these columns. `_x` columns refer to the columns which got added by the first join (hence they refer to the `ORIGIN` airport), and the `_y` columns refer to the `DEST` airport.

#### Step 2

In [25]:
summary_stats = large_route_stats.groupby(['ORIGIN', 'municipality_x', 'DEST', 'municipality_y'])['PASSENGERS', 'FREIGHT'].sum()
summary_stats.head(10)

PASSENGERS  FREIGHT
ORIGIN municipality_x DEST municipality_y                     
ABQ    Albuquerque    ABQ  Albuquerque              0     1655
                      AFW  Fort Worth             227        0
                      AMA  Amarillo                22      171
                      ANC  Anchorage                0        0
                      ATL  Atlanta             132649    31513
                      AUS  Austin               45961     6360
                      BDL  Hartford                64        0
                      BHM  Birmingham              34        0
                      BNA  Nashville             1908     9669
                      BOI  Boise                  306       33

A bunch of information to unpack here, so try to understand each point properly.

- The `groupby()` method in the `DataFrame` class executes the group by transformation.
- The argument to `groupby` is the list of columns by whom you want to group. In our case, we chose the `ORIGIN` and `DEST` columns. We also included the municipality columns to get readable names for the airports.
    - These columns will become part of the (multi)index in the output of group by.
- The return value of the `groupby()` method is not a `DataFrame`, but a type called `DataFrameGroupBy`. It is expected that the user will run an aggregate function on it.
- You can choose some columns from the dataframe which you want to be aggregated. We have chosen `PASSENGERS` and `FREIGHT` columns, because that's what we want to measure.
   - This part is optional. If you don't specify that list, `pandas` will aggregate all columns.
- Finally, you provide the aggregate function. In this case, we want the sum of numbers so we call the `sum()` method.

#### Step 3

This step is self-explanatory.

In [26]:
summary_stats.sort_values(by='FREIGHT', ascending=False)

PASSENGERS    FREIGHT
ORIGIN municipality_x    DEST municipality_y                          
ANC    Anchorage         SDF  Louisville                  0  455897457
                         CVG  Cincinnati                  0  378488076
LAX    Los Angeles       MEM  Memphis                 52293  249759069
ANC    Anchorage         ORD  Chicago                 76637  230537825
LAX    Los Angeles       HNL  Honolulu              1148693  170798479
ANC    Anchorage         LAX  Los Angeles             72506  155891164
EWR    Newark            MEM  Memphis                 41910  155157363
OAK    Oakland           MEM  Memphis                  3621  146433056
MEM    Memphis           LAX  Los Angeles             53382  144300307
                         EWR  Newark                  40988  143671754
ANC    Anchorage         MEM  Memphis                     0  130819244
HNL    Honolulu          LAX  Los Angeles           1154762  129977645
SDF    Louisville        ANC  Anchorage                   0  129888723
MEM    Memphis           OAK  Oakland                  3609  126178800
ONT    Ontario           SDF  Louisville                 50  121092455
MEM    Memphis           ANC  Anchorage                   7  120649056
ONT    Ontario           MEM  Memphis                     0  116171862
DFW    Dallas-Fort Worth SDF  Louisville             115124  112364542
MEM    Memphis           SEA  Seattle                     7  103335937
CVG    Cincinnati        ANC  Anchorage                   0  102418498
SDF    Louisville        EWR  Newark                  42016  101390569
MEM    Memphis           MIA  Miami                   25043  100872450
SDF    Louisville        ONT  Ontario                    17   98241056
MIA    Miami             MEM  Memphis                 23197   96666068
EWR    Newark            SDF  Louisville              43314   96167376
SDF    Louisville        PHL  Philadelphia            42614   94122490
LAX    Los Angeles       SFO  San Francisco         2000078   91402022
MEM    Memphis           DEN  Denver                 100905   91222137
SDF    Louisville        DFW  Dallas-Fort Worth      114352   89783078
ORD    Chicago           MEM  Memphis                146296   88922572
...                                                     ...        ...
PHL    Philadelphia      GEG  Spokane                    36          0
                         GPT  Gulfport                 1007          0
DTW    Detroit           RNO  Reno                      485          0
EWR    Newark            BOI  Boise                      43          0
                         CAE  Columbia                    0          0
                         GSP  Greenville              42707          0
                         CID  Cedar Rapids              119          0
                         GRB  Green Bay                 153          0
                         GEG  Spokane                    24          0
                         FWA  Fort Wayne               4665          0
                         EWR  Newark                      0          0
                         DSM  Des Moines              12433          0
                         DOV  Dover                       0          0
                         DAY  Dayton                  23731          0
                         CRP  Corpus Christi              0          0
PHF    Newport News      AGS  Augusta                     1          0
                         BHM  Birmingham                 11          0
                         DCA  Washington                 13          0
EWR    Newark            CHA  Chattanooga             16460          0
PHF    Newport News      FLL  Fort Lauderdale           148          0
                         IAD  Washington                 26          0
                         JFK  New York                    0          0
                         MGM  Montgomery                  0          0
                         ORF  Norfolk                     4          0
  

### Problem 2

List all routes which are serviced by at least 5 different airlines. A route is called serviced by an airline if more than 1,000 passengers fly that airline on that route on _any_ month of the year.

- This problem also requires us to group by `ORIGIN` and `DEST` columns.
- In any group that's created, we now have to find how many carriers fly a mininum number of passengers. This is something that an existing aggregate function can't do.
- This problem illustrates how we can use custom aggregate functions

The syntax for group by call remains pretty much the same, except that instead calling an aggregate function, you call the `apply()` method, as shown below.

```python
my_df.groupby(['group_col1', 'group_col2']).apply(lambda x: my_agg_function(x))
```
The most important thing to understand here is: What would be value (or even the type) of `x`, which is the argument to our custom function.

- `x` is of type `DataFrame`, which contains has the same index type and the same list of columns as `my_df`.
- For example, if we grouped by `large_route_stats` by `ORIGIN` and `DEST`, we would get groups like (`JFK`, `ATL`) or (`LAX`, `SFO).
   - In the dataframe, there are a number of rows which belong to any one group - as there are multiple airlines which service the route, there is data for different months on that route, etc.
   - The value of `x` will contain only those rows which belong to that group.
   
Let's see this in action.

In [114]:
def print_group_details(group_df):
    print("\n---New Group ----\n")
    print(group_df[['ORIGIN', 'DEST', 'MONTH', 'CARRIER_NAME', 'PASSENGERS', 'FREIGHT']])

In [115]:
large_route_stats[large_route_stats['ORIGIN'] == 'JFK'].groupby(['ORIGIN', 'DEST']).apply(print_group_details)


---New Group ----

      ORIGIN DEST  MONTH          CARRIER_NAME  PASSENGERS  FREIGHT
66588    JFK  ABQ      6  Alaska Airlines Inc.         137        0
66589    JFK  ABQ      1       JetBlue Airways        2337        0
66590    JFK  ABQ      2       JetBlue Airways        2597        0
66591    JFK  ABQ      9       JetBlue Airways        2828        0
66592    JFK  ABQ     11       JetBlue Airways        3014        0
66593    JFK  ABQ     12       JetBlue Airways        3305        0
66594    JFK  ABQ     10       JetBlue Airways        3625        0
66595    JFK  ABQ      3       JetBlue Airways        3668        0
66596    JFK  ABQ      4       JetBlue Airways        3693        0
66597    JFK  ABQ      8       JetBlue Airways        3745        0
66598    JFK  ABQ      6       JetBlue Airways        3867        0
66599    JFK  ABQ      5       JetBlue Airways        3954        0
66600    JFK  ABQ      7       JetBlue Airways        3977        0

---New Group ----

      OR

137029    JFK  LAS      7        Delta Air Lines Inc.       24802    18864

---New Group ----

      ORIGIN DEST  MONTH                     CARRIER_NAME  PASSENGERS  FREIGHT
29024    JFK  LAX      1                   Atlas Air Inc.           0   141610
29025    JFK  LAX     11      Federal Express Corporation           0   173089
29026    JFK  LAX     12      Federal Express Corporation           0   503611
29027    JFK  LAX      3                  Sky Lease Cargo           0    61500
29028    JFK  LAX      2  Polaris Aviation Solutions, LLC          13        0
29029    JFK  LAX     12             Delta Air Lines Inc.          57        0
29030    JFK  LAX      4             Delta Air Lines Inc.          65        0
29031    JFK  LAX      6            United Air Lines Inc.         108    17337
29032    JFK  LAX      9             Alaska Airlines Inc.       15623      426
29033    JFK  LAX     11             Alaska Airlines Inc.       18530       70
29034    JFK  LAX     12            

""


- Our custom function simply prints some of the columns of the dataframe provided to us in the argument.
- As you can see, we are trying to find all routes that originate from JFK.
- The print output shows what each group looks like, i.e., each group is a `DataFrame` with the same `ORIGIN` and `DEST`, containing all the rows which correspond to a specific set of values for `ORIGIN` and `DEST`.

Now let's focus on solving the problem 2. Our main task is to define the custom aggregation function.

- First thing to understand is that our custom aggregate function is just any function which gets a `DataFrame` as input. We can do any transformation to that dataframe in order to get our answer.
- In this case, we need to filter that dataframe by the condition that the number of passengers > 1000.
- After the filtering, we will get a new dataframe whose `CARRIER_NAME` column contains all the airline carriers which satisfy the condition stated in the problem.
- Using `unique()` on that column, we can find the carriers servicing that route with at least some passengers.

Here is the implementation:

In [116]:
def num_carriers_service(group_df):
    serviced_rows = group_df[group_df['PASSENGERS'] > 1000]
    unique_carriers = list(serviced_rows['CARRIER_NAME'].unique())
    
    # We are returning a series to get two columns in the output - one containing the number of carriers,
    # and other containing the actual list of carriers.
    return pd.Series([len(unique_carriers), str(unique_carriers)], index=['num_carriers', 'carriers'])

In [117]:
route_carriers = large_route_stats.groupby(['ORIGIN', 'municipality_x', 'DEST', 'municipality_y']).apply(num_carriers_service)
route_carriers.head(10)

num_carriers  \
ORIGIN municipality_x DEST municipality_y                 
ABQ    Albuquerque    ABQ  Albuquerque                0   
                      AFW  Fort Worth                 0   
                      AMA  Amarillo                   0   
                      ANC  Anchorage                  0   
                      ATL  Atlanta                    1   
                      AUS  Austin                     3   
                      BDL  Hartford                   0   
                      BHM  Birmingham                 0   
                      BNA  Nashville                  0   
                      BOI  Boise                      0   

                                                                                    carriers  
ORIGIN municipality_x DEST municipality_y                                                     
ABQ    Albuquerque    ABQ  Albuquerque                                                    []  
                      AFW  Fort Worth                                                     []  
                      AMA  Amarillo                                                       []  
                      ANC  Anchorage                                                      []  
                      ATL  Atlanta                                  ['Delta Air Lines Inc.']  
                      AUS  Austin          ['Allegiant Air', 'Southwest Airlines Co.', 'F...  
                      BDL  Hartford                                                       []  
                      BHM  Birmingham                                                     []  
                      BNA  Nashville                                                      []  
                      BOI  Boise                                                          []

In [118]:
route_carriers[route_carriers['num_carriers'] >= 5]

num_carriers  \
ORIGIN municipality_x DEST municipality_y                 
ABQ    Albuquerque    DEN  Denver                     6   
                      IAH  Houston                    5   
                      ORD  Chicago                    6   
ATL    Atlanta        DCA  Washington                 5   
                      DEN  Denver                     7   
                      IAH  Houston                    7   
                      LAX  Los Angeles                5   
                      LGA  New York                   5   
                      MCO  Orlando                    5   
                      ORD  Chicago                    8   
                      PHL  Philadelphia               5   
AUS    Austin         DEN  Denver                     6   
                      LAX  Los Angeles                6   
                      ORD  Chicago                    5   
                      SFO  San Francisco              5   
BDL    Hartford       ORD  Chicago                    8   
BHM    Birmingham     ORD  Chicago                    6   
BNA    Nashville      BOS  Boston                     5   
                      DEN  Denver                     6   
                      IAH  Houston                    5   
                      LGA  New York                   6   
                      MSP  Minneapolis                6   
                      ORD  Chicago                    8   
                      PHL  Philadelphia               5   
BOS    Boston         BNA  Nashville                  6   
                      JFK  New York                   8   
                      LAX  Los Angeles                6   
                      LGA  New York                   5   
                      PIT  Pittsburgh                 5   
                      RDU  Raleigh/Durham             6   
...                                                 ...   
SFO    San Francisco  JFK  New York                   5   
                      LAS  Las Vegas                  5   
                      LAX  Los Angeles                8   
                      PDX  Portland                   7   
                      PHX  Phoenix                    5   
                      SAN  San Diego                  5   
                      SEA  Seattle                    7   
                      SLC  Salt Lake City             5   
SJC    San Jose       DEN  Denver                     5   
                      LAX  Los Angeles                5   
                      SEA  Seattle                    5   
SLC    Salt Lake City DEN  Denver                     8   
                      IAH  Houston                    5   
                      LAS  Las Vegas                  5   
                      LAX  Los Angeles                5   
                      PHX  Phoenix                    6   
                      SFO  San Francisco              5   
SMF    Sacramento     LAX  Los Angeles                6   
                      SEA  Seattle                    5   
STL    St Louis       DEN  Denver                     7   
                      LGA  New York                   5   
                      MSP  Minneapolis                5   
                      ORD  Chicago                    9   
SYR    Syracuse       DTW  Detroit                    5   
                      ORD  Chicago                    5   
TPA    Tampa          MSP  Minneapolis                5   
                      RDU  Raleigh/Durham             5   
TUL    Tulsa          DEN  Denver                     7   
                      ORD  Chicago                    7   
TYS    Knoxville      ORD  Chicago                    6   

                                                                                    carriers  
ORIGIN municipality_x DEST municipality_y                                                     
ABQ    Albuquerque    DEN  Denver          ['Trans States Airlines', 'Frontier Airlines I...  
                      IAH  Houston      

Spend some time with the solution and explanation for this problem. It's required to solve the exercise below :-)

### Exercise

Solve problem 2 above but with a new criteria - an airline is considered as servicing the route if it transports atleast 50,000 passengers in a year.

## Exercise set 1

This set uses the same data as what we used for the lecture examples.

1. For each elevation range 0-500 ft, 500-1000 ft, 1000-2000 ft, 2000-5000 ft and 5000-above ft, find the number of passengers flying out of airports which lie at that elevation.

2. Find the number of passengers and the amount of freight for each airport type in the US (heliport, small_airport, seaplane_base, balloonport, medium_airport, large_airport)

3. In every route, the number of passengers going forward in the route is not the same as the number of passengers going back. Let's say that people are migrating in one of the directions which has higher passengers. Find the routes with the largest amount of migration.

4. Same as 3, but for freight. In this case, it's likely even more the case that more freight is transferred in one direction than the opposite.

5. The busiest airports might change over the course of the year, due to changes in weather. Find the top 5 busiest airports in the US (measured by passenger travelling in to there) for each month of the year.

## Exercise set 2

For this set, we will use the date in the file [`video_game_sales.csv`](https://raw.githubusercontent.com/amangup/data-analysis-bootcamp/master/08-Pandas3/video_game_sales.csv). This file contains the sales, ratings and metadata for most of the video games ever developed.

1. Find the top 10 publishers in the game industry by total global sales in the last 10 years.

2. For each genre, count how many games of that genre a publisher has made (there should be a column for each publisher). Limit this to top 10 publishers found in 1.

3. Same as in 2, but this time count how many highly rated games of each genre a publisher has made.
  - Highly rated is defined as follows: Across all games in the last 10 years, find the 80th percentile critic score and 80th percentile user score. All games whose critic score **and** the user score are higher than the 80th percentile scores are highly rated. If ratings are not provided, ignore that game.

4. For each publisher, how many unique developers have they worked with.

5. For each publisher, who is their most preferred developer? This is simply the developer who the publisher has worked with most often in the last 10 years.

6. For each publisher, who is their most successful developer by total global sales in the last 10 years?

7. For each publisher, find the total sales in each region, as well as global sales, for each game platform (there should be a column for each game platform). Sort the columns by total global sales of each platform.

8. For each value of age rating (`Rating` column), find the average critic score, average user score, and total sales in each region and globally.
